In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
%cd "/content/drive/My Drive/NLA/Project/RankQA"
#%cd "/RankQA/"

/content/drive/My Drive/NLA/Project/RankQA


In [6]:
!ls

evaluate.py	   __pycache__	README.md	  run.py
pre-trained-model  ranker	requirements.txt


In [0]:
import argparse
import logging
import os
import sys
import time
import uuid

import numpy as np
import torch
from evaluate import Evaluator
from ranker import data_utils, data_loader
from ranker import ranker_net
from ranker.data_utils import batchify_pair

np.random.seed(42)
torch.manual_seed(1)
torch.cuda.manual_seed(1)

logger = logging.getLogger()

In [0]:
class Argument:
# subsampling
    max_per_q = 2
    valid_split = 0.9
    stratify_valid = True
    max_depth = 2
# fixed stuff
    cuda = True
# learning
    epochs =  100
    lr = 0.005
    batch_size = 256

# other
    name = 'regular_WM'
# tuning
    linear_dim = 512
    reg = 0.00005



In [0]:
# ---------------------------------------------------------------------------------------------
# description of features to use during training
# ---------------------------------------------------------------------------------------------
SQUAD_path = "/content/drive/My Drive/NLA/Project/SQUAD"
WIKI_path = "/content/drive/My Drive/NLA/Project/WIKI"

feature_descriptors = [
    {'feature_name': 'sum_span_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'sum_doc_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'doc_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'span_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'min_doc_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'max_doc_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'avg_doc_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'max_span_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'min_span_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'avg_span_score', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'first_occ', 'normscheme': 'minmax', 'preprocess': np.log},
    {'feature_name': 'num_occ', 'normscheme': 'minmax', 'preprocess': np.log},
    #{'feature_name': 'context_len', 'normscheme': 'minmax', 'preprocess': np.log},
    #{'feature_name': 'question_len', 'normscheme': 'minmax', 'preprocess': np.log}
]

train_filesSQ = [os.path.join(SQUAD_path,
                              'SQuAD-v1.1-train-default-pipeline.preds-part-{}'.format(i)) for i in range(88)]

'''train_filesWM = [os.path.join(WIKI_path,
                              'WikiMovies-train-default-pipeline.preds-part-{}'.format(i)) for i in range(96)]

train_filesWQ = [os.path.join('features',
                              'WebQuestions-train-default-pipeline.preds-part-{}'.format(i)) for i in range(4)]

train_filesT = [os.path.join('features',
                             'CuratedTrec-train-default-pipeline.preds-part-{}'.format(i)) for i in range(2)]'''

test_files = {

      'SQuAD': [os.path.join(SQUAD_path,
                           'SQuAD-v1.1-dev-default-pipeline.preds-part-{}'.format(i)) for i in range(11)]
              
     
    }

''''SQuAD': [os.path.join(SQUAD_path,
                           'SQuAD-v1.1-dev-default-pipeline.preds-part-{}'.format(i)) for i in range(11)]
                           
   'WikiMovies': [os.path.join(WIKI_path,
                                'WikiMovies-test-default-pipeline.preds-part-{}'.format(i)) for i in range(10)]

                           '''

train_files = [train_filesSQ]

# -------------------
# Arguments
# -------------------


def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


args_ = Argument()

In [0]:
def val(valid_loader_, model_, args_):
    valid_loss = []
    for i, input in enumerate(valid_loader_):
        inl, inr, target = input
        l = model_.eval_pairwise(inl, inr, target)
        valid_loss.append(l)
    return np.mean(valid_loss)


def train(train_loader_, valid_loader_, model_, args_, evaluators_, modelfilename_):
    initiallywrong = []
    for evaluator_ in evaluators_:
        log, wrongansw = evaluator_.evaluate(model_)
        initiallywrong.append(wrongansw)
        logger.info(log)
    best_val_loss = float('inf')
    best_val_iteration = 0

    for b in range(args_.epochs):
        loss = []
        logger.info('==================== EPOCH {} ================================'.format(b))
        for i, input in enumerate(train_loader_):
            inl, inr, target = input
            l = model_.update_pairwise(inl, inr, target)
            loss.append(l)
        val_loss = val(valid_loader_, model_, args_)
        logger.info('Epoch finished avg loss = {}'.format(np.mean(loss)))
        logger.info('Validation loss = {}'.format(val_loss))

        # check if
        if best_val_loss > val_loss:
            # save model
            logger.info('BEST EPOCH SO FAR --> safe model')
            model_.safe(modelfilename_)
            best_val_loss = val_loss
            best_val_iteration = 0
        best_val_iteration += 1
        if best_val_iteration > 10:
            # stop training
            logger.info("EARLY STOPPING")
            break
    model_.load(modelfilename_)
    # evaluate
    for i, evaluator_ in enumerate(evaluators_):
        log, wrong = evaluator_.evaluate(model_)
        logger.info(log)
        logger.info('fraction still correct = {}'.format(len(np.intersect1d(initiallywrong[i], wrong)) /
                                                         len(wrong)))


In [11]:
    output_path = "/content/drive/My Drive/NLA/Project/output/"

    logger.setLevel(logging.INFO)
    fmt = logging.Formatter('%(asctime)s: [ %(message)s ]', '%m/%d/%Y %I:%M:%S %p')
    console = logging.StreamHandler()
    console.setFormatter(fmt)
    logger.addHandler(console)
    ts = time.strftime("%Y%m%d-") + str(uuid.uuid4())[:8]

    logfilename = os.path.join(output_path, args_.name + ts + ".txt")
    modelfilename = os.path.join(output_path, args_.name + ts + ".mdl")
    logfile = logging.FileHandler(logfilename, 'w')
    logfile.setFormatter(fmt)
    logger.addHandler(logfile)
    logger.info('COMMAND: %s' % ' '.join(sys.argv))

    logger.info(args_)



05/01/2020 09:35:17 AM: [ COMMAND: /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-967954f9-2920-48b6-8c7e-934d3e6e707f.json ]
05/01/2020 09:35:17 AM: [ <__main__.Argument object at 0x7fd4f3c7d9b0> ]


In [12]:
    # load training data
    logger.info('Loading {} files with training samples..'.format(sum(len(x) for x in train_files)))
    train_data, valid_data, normalizers = data_utils.load_subsample(train_files, feature_descriptors, args_)
    logger.info('Done. Number of train pairs loaded: {} (valid = {})'.format(len(train_data), len(valid_data)))


05/01/2020 09:35:19 AM: [ Loading 88 files with training samples.. ]
05/01/2020 09:35:19 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-0 ]
05/01/2020 09:35:21 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-1 ]
05/01/2020 09:35:24 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-2 ]
05/01/2020 09:35:26 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-3 ]
05/01/2020 09:35:28 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-4 ]
05/01/2020 09:35:30 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/SQuAD-v1.1-train-default-pipeline.preds-part-5 ]
05/01/2020 09:35:32 AM: [ loading and processing /content/drive/My Drive/NLA/Project/SQUAD/

In [13]:
    # load dev data
    evaluators = []
    for dataset in test_files:
        logger.info('Loading {} dataset with {} files..'.format(dataset, len(test_files[dataset])))
        test_data = data_utils.load_full(test_files[dataset])
        test_dataset = data_utils.build_validation_dataset(test_data, normalizers)
        evaluator = Evaluator(test_dataset[0], test_dataset[1], dataset)
        evaluators.append(evaluator)



05/01/2020 09:43:03 AM: [ Loading SQuAD dataset with 11 files.. ]


In [1]:
test_files

NameError: ignored

In [14]:
    # generate train data loader
    logger.info('Initialize training loader..')
    train_dataset = data_loader.PairwiseRankingDataSet(train_data, normalizers)
    valid_dataset = data_loader.PairwiseRankingDataSet(valid_data, normalizers)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args_.batch_size,
        sampler=torch.utils.data.sampler.RandomSampler(train_dataset),
        pin_memory=args_.cuda,
        collate_fn=batchify_pair
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=args_.batch_size,
        sampler=torch.utils.data.sampler.RandomSampler(valid_dataset),
        pin_memory=args_.cuda,
        collate_fn=batchify_pair
    )

    # init model
    logger.info('Init model..')
    model = ranker_net.RankerNet(args_, train_dataset.num_feat)
    logger.info('Done.')

    

05/01/2020 09:44:29 AM: [ Initialize training loader.. ]
05/01/2020 09:44:38 AM: [ Init model.. ]
05/01/2020 09:44:48 AM: [ Done. ]


In [0]:
train_dataset.y

In [0]:
for b in range(args.epochs):
        loss = []
        logger.info('==================== EPOCH {} ================================'.format(b))
        for i, input in enumerate(train_loader):
            inl, inr, target = input
            l = model.update_pairwise(inl, inr, target)
            loss.append(l)
        val_loss = val(valid_loader, model, args)
        logger.info('Epoch finished avg loss = {}'.format(np.mean(loss)))
        logger.info('Validation loss = {}'.format(val_loss))

        # check if
        if best_val_loss > val_loss:
            # save model
            logger.info('BEST EPOCH SO FAR --> safe model')
            model.safe(modelfilename)
            best_val_loss = val_loss
            best_val_iteration = 0
        best_val_iteration += 1
        if best_val_iteration > 10:
            # stop training
            logger.info("EARLY STOPPING")
            break
    model.load(modelfilename)


IndentationError: ignored

In [0]:
'''baseline_performance = 0
top_performance = 0
model_performance = 0
wrong = []
with torch.no_grad():
   inputs = []
   solvable = False'''
for i, x in enumerate(test_dataset[0]):
  inputs = []
  for j, candidate in enumerate(test_dataset[0][i]):
    inputs.append(candidate)

#print(inputs)
  scores = model.predict(torch.stack(inputs))
  print(scores)
#test_dataset[1][0]


In [15]:
# kick off training
   train(train_loader, valid_loader, model, args_, evaluators, modelfilename)
   for handler in logger.handlers:
        handler.flush()


0it [00:00, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of cuda is deprecated:
	cuda(torch.device device, bool async, *, torch.memory_format memory_format)
Consider using one of the following signatures instead:
	cuda(torch.device device, bool non_blocking, *, torch.memory_format memory_format)
10570it [00:02, 3569.97it/s]
05/01/2020 09:45:01 AM: [ SQuAD evaluation results w/o re-ranking: 0.30018921475875116 / upper bound: 0.5428571428571428; rank-qa: 0.23765373699148534 ]
05/01/2020 09:45:01 AM: [ ==================== EPOCH 0 ================================ ]
05/01/2020 09:45:02 AM: [ Epoch finished avg loss = 0.10649658653772238 ]
05/01/2020 09:45:02 AM: [ Validation loss = 0.10606715514471657 ]
05/01/2020 09:45:02 AM: [ BEST EPOCH SO FAR --> safe model ]
05/01/2020 09:45:03 AM: [ ==================== EPOCH 1 ================================ ]
05/01/2020 09:45:03 AM: [ Epoch finished avg loss = 0.09985008149436025 ]
05/01/2020 09:45:03 AM: [

In [0]:
import numpy as np
import torch
from tqdm import tqdm


In [0]:
print(test_dataset[1][0:10])

In [0]:
        model = model_eval
        X = test_dataset[0]
        y = test_dataset[1]
        y_pred = []
        baseline_performance = 0
        top_performance = 0
        model_performance = 0
        wrong = []
        with torch.no_grad():
            for i, x in tqdm(enumerate(X[0:40])):
                inputs = []
                solvable = False
                for j, candidate in enumerate(x):
                    
                    if y[i][j] == 1:
                        solvable = True
                        print(solvable)
                    inputs.append(candidate)

                baseline_performance += int(y[i][0])

                if not solvable:
                    # no need to predict unsolvable questions
                    wrong.append(i)
                    continue

                top_performance += 1

                scores = model.predict(torch.stack(inputs))

                y_pred.append(np.argmax(scores))
                for k in range(len(x)):
                  if(test_data[i][k]['target'] == 1):
                    actual_index = k
                    print('QuestionID: ' + str(test_data[i][j]['qid']) +  ' predicted index : ' + str(np.argmax(scores)) + ' actual index: ' + str(k) )
                    break;
                print((scores[0:10]))
                model_performance += int(y[i][np.argmax(scores[0:10])])
                if int(y[i][np.argmax(scores)]) == 0:
                    wrong.append(i)

        baseline_performance = baseline_performance / len(y)
        model_performance = model_performance / len(y)
        top_performance = top_performance / len(y)
        log = '{} evaluation results w/o re-ranking: {} / upper bound: {}; rank-qa: {}'.format('SQUAD',
                                                                                               baseline_performance,
                                                                                               top_performance,
                                                                                               model_performance)


In [0]:
for i in range(len(y_pred)):
  x = y_pred[i]
  for j in range(len(test_data[i])):
    if(test_data[i][j]['target'] == 1):
      ac = j
      break;
  print('predicted: ' + str(y_pred[i]) + ': actual: ' + str(ac))

In [0]:
    initiallywrong = []
    for evaluator_ in evaluators:
        log, wrongansw = evaluator_.evaluate(model)
        initiallywrong.append(wrongansw)
        logger.info(log)
    best_val_loss = float('inf')
    best_val_iteration = 0

9952it [00:02, 3384.76it/s]
04/15/2020 12:00:52 AM: [ WikiMovies evaluation results w/o re-ranking: 0.3867564308681672 / upper bound: 0.6498191318327974; rank-qa: 0.21764469453376206 ]


In [0]:
wrong = []
log, wrongans = evaluators[0].evaluate(model)
wrong.append(wrongans)

9952it [00:02, 3545.37it/s]


In [0]:
print(len(wrongans))
print(len(wrongansw))
print(len(np.intersect1d(wrongans, wrongansw)))

5651
7793
5306
